In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from botasaurus import *
import os
import time
import requests
from src.constants import zona_prop_url

In [ ]:
@request
def scrap_zonaprop(request: AntiDetectRequests, link: str):
    # headers = {'User-Agent': 'Mozilla/5.0'}
    soup = request.bs4(link, 
                        #    headers=headers
                           )
    # print(response)
    # soup = BeautifulSoup(response.content, 'html.parser')
    # print(soup)
    # Initialize the lists to store the data
    prices = []
    locations = []
    addresses = []
    features = []
    descriptions = []
    expensas = []
    ap_links = []

    # Start scraping from the first page
    while True:
        # Select all the div elements with the required class
        property_elements = soup.select('div.sc-1tt2vbg-4.dFNvko')

        # Iterate over each property element
        for property_element in property_elements:
            # Find the child elements with the required data-qa attributes
            price_element = property_element.select_one('div[data-qa="POSTING_CARD_PRICE"]')
            location_element = property_element.select_one('div[data-qa="POSTING_CARD_LOCATION"]')
            address_element = property_element.select_one('div[class="sc-ge2uzh-0 eXwAuU"]')
            features_elements = property_element.select('div[data-qa="POSTING_CARD_FEATURES"] span')
            description_element = property_element.select_one('div[data-qa="POSTING_CARD_DESCRIPTION"]')
            expensas_element = property_element.select_one('div[data-qa="expensas"]')
            ap_link_element = property_element.select_one('div[data-to-posting]')

            # Extract the text from the elements and append it to the lists
            prices.append(price_element.text if price_element else np.nan)
            locations.append(location_element.text if location_element else np.nan)
            addresses.append(address_element.text if address_element else np.nan)
            features.append([feature_element.text for feature_element in features_elements])
            descriptions.append(description_element.text if description_element else np.nan)
            expensas.append(expensas_element.text if expensas_element else np.nan)
            ap_links.append(ap_link_element.text if ap_link_element else np.nan)

        # Check if there is a next page
        next_page = soup.select_one('a[data-qa="PAGING_NEXT"]')
        if next_page and next_page['href']:
            # Prepend the base URL to the href
            full_url = 'https://www.zonaprop.com.ar/' + next_page['href']
            # Go to the next page
            soup = request.bs4(full_url,
                                    # headers=headers
                                    )
            print(next_page['href'])
        else:
            # No more pages, break the loop
            break
    
    # Create a DataFrame from the lists
    return {
        'Price': prices,
        'Location': locations,
        'Address': addresses,
        'Features': features,
        'Description': descriptions,
        'Expensas': expensas,
        'Link': ap_links
    }

# Call the function with the URL of the website you want to scrape
df = scrap_zonaprop('https://www.zonaprop.com.ar/inmuebles-alquiler-capital-federal.html')


In [22]:
df = pd.DataFrame(df)

,Price,Location,Address,Features,Description,Expensas
0,USD 1.700,"Palermo Chico, Palermo",Castex 3400,"[ 82 m² , 82 m² , 82 m² , 82 m² , 3 amb. ,...",Precio paquete con todo incluido! Espectacular...,NaN
1,USD 640,"Barracas, Capital Federal",Rocdale 1200,"[ 180 m² , 180 m² , 160 m² , 160 m² , 3 am...",Depósito desarrollado en una planta. Apto todo...,NaN
2,$ 120.000,"Retiro, Capital Federal",Viamonte al 700,"[ 152 m² , 152 m² , 152 m² , 152 m² , 5 am...",Salaya Romera Propiedades Ofrece a la Venta un...,$ 140.000 Expensas
3,USD 6.720,"Monserrat, Capital Federal",Corrientes al 300 (mq) - 423,"[ 408 m² , 408 m² , 8 baños , 8 baños , 3 ...",¡¡oficina comercial!! Todo el piso Piso recién...,$ 480.000 Expensas
4,Consultar precio,"Parque Patricios, Capital Federal",Zabaleta 85,"[ 4636 m² , 4636 m² , 4636 m² , 4636 m² ]","Sobre lote de 1166, 00Total Construido 4636, 0...",NaN
...,...,...,...,...,...,...
17944,$ 700.000,"Recoleta, Capital Federal",Peña 2000 2°,"[ 75 m² , 75 m² , 70 m² , 70 m² , 3 amb. ,...","Excelente tres ambientes Recoleta, un departam...",$ 85.000 Expensas
17945,USD 600,"Recoleta, Capital Federal",Larrea 1100,"[ 85 m² , 85 m² , 76 m² , 76 m² , 3 amb. ,...",Amplio department ubicado en inmejorable ubica...,$ 51.000 Expensas
17946,$ 150.000,"Liniers, Capital Federal",AV RIVADAVIA 11000. Entre Martiniano leguizamo...,"[ 30 m² , 30 m² , 30 m² , 30 m² , 1 baño ,...",Xintel(fde-fde-930) Alquiler de Local en Linie...,$ 25.000 Expensas
17947,$ 500.000,"Villa Devoto, Capital Federal",AV. SAN MARTIN 6900. Entre Mosconi y Carlos a....,"[ 70 m² , 70 m² , 70 m² , 70 m² , 1 baño ,...",Local alquiler Villa Devoto. Amplio local en a...,NaN
